In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from sqlalchemy import create_engine
# 创建浏览器
driver = webdriver.Chrome()
url = "https://www.51job.com/"
driver.get(url)
wait = WebDriverWait(driver, 10) # 用来判断是否有点击事件，如果规定的事件之内没有点击事件则报错
data = []  # 数据列表
details_links = []   # 详情链接列表
search_btn = driver.find_element_by_css_selector("#kwdselectid")
search_btn.send_keys("数据分析")
search = ["body > div.content > div > div.fltr.radius_5 > div > button", "#filter_issuedate > ul > li:nth-child(3) > a"]
# 用于实现数据分析搜索和前三天搜索的功能实现
for i in search:
    confirm_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, i )))
    confirm_btn.click()
    time.sleep(3)  # 表示等待3秒进行其它操作
n = 8  # 这里的8事第一页中下一页的selector中的一部分，每一页的下一页的selector都递增
for j in range(5): 
    print("正在爬取第"+str(j+1)+"页的信息")
    # 解析网页
    soup =BeautifulSoup(driver.page_source, "lxml")
    # find()返回的是一个结果，而find_all()返回的是一个列表
    soup_findall = soup.find("div", class_ = "dw_table").find_all("div", class_ = "el")
    # 循环提取信息
    # 确定是否又下一页的按钮
    c = "#resultList > div.dw_page > div > div > div > ul > li:nth-child({n_n}) > a".format(n_n = n)
    confirm_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, c)))  # 用于判断下一页是否可以点击
    dic = {}
    for i in soup_findall[1:]:
      job_title = i.find("p", class_ = "t1").a["title"]  # a["title"]表示a标签下title属性的值
      details_link = i.find("p", class_ = "t1").a["href"] # 详情连接
      company_name = i.find("span", class_ = "t2").a["title"]
      working_place = i.find("span", class_ = "t3")
      salary = i.find("span", class_ = "t4").text
      release_time = i.find("span", class_ = "t5") .text  # .text表示text文本        
      details_links.append(details_link)      
      dic = {"职位名": job_title, "公司名":company_name, "工作地点":working_place, "薪资":salary, "发布时间": release_time}
      data.append(dic)
    n = n + 1
    # 点击下一页
    print("第"+str(j+1)+"页信息爬取完毕")
    confirm_btn.click()
data1 = pd.DataFrame(data) 
data1

正在爬取第1页的信息
第1页信息爬取完毕
正在爬取第2页的信息
第2页信息爬取完毕
正在爬取第3页的信息
第3页信息爬取完毕
正在爬取第4页的信息
第4页信息爬取完毕
正在爬取第5页的信息
第5页信息爬取完毕


,职位名,公司名,工作地点,薪资,发布时间
0,专业服务助理(投标/行业研究和数据分析支持),毕马威中国 KPMG China,[南京],4.5-5千/月,07-22
1,液奶南京区域配送中心数据分析专员（物流）,内蒙古伊利实业集团股份有限公司,[南京],4.5-6千/月,07-22
2,数据分析师,江苏视界互联传媒有限公司,[南京-玄武区],15-25万/年,07-22
3,大数据分析 集团研发(J197488),南京我爱我家房屋租赁置换有限公司,[南京-鼓楼区],1.5-2万/月,07-22
4,数据分析师,江苏微皓智能科技有限公司,[南京-建邺区],6-8千/月,07-22
...,...,...,...,...,...
245,数据开发工程师（环保事业部）,南京瀚和软件技术有限公司,[南京-鼓楼区],1-1.5万/月,07-22
246,微生物检验咨询师（临床检验/微生物/免疫专业）,江苏吉诺思美精准医学科技有限公司,[南京],0.8-1.2万/月,07-22
247,销售顾问（互联网+五险一金+带薪培训）,客如云科技（北京）股份有限公司,[南京],1-1.5万/月,07-22
248,高级软件工程师,南京海司唯尔信息科技有限公司,[南京],1.5-2万/月,07-22
